## 시설물 유지보수대장 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 유지보수대장 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 유지보수대장 목록/selectIoFmCwkRegsList.csv'

In [2]:
# 개별 시설물 유지보수대장목록 수
g_totalCount = 0

In [3]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmCwkRegsList'

In [4]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [5]:
JSONKEY = "items"
DUMMY = 0

## 시설물 유지보수대장 목록 전용 스크랩 함수

In [13]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                
                # 25번 도로시설물 관리현황 정보의 검색결과 패턴을 비교하였을때 items에만 결과를 줄 것으로 예상  
                # 여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
                if(response.json()["response"]["body"].get(jsonkey) == None):                  
                    print(jsonkey+"is None !!!!")
                    break
                

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"][jsonkey]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        print(rowData)
                        emptyPd = emptyPd.append(rowData)
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [14]:

numOfRows = 1000
PAGEYN=0

In [15]:
class noDataFound(Exception): pass  # declare a label


In [16]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()
i = 0
try:
    while True:
        pageNo = 1
        global g_totalCount
        g_totalCount = 0
        print("i=",i)
        i+=1
        while True:
            # 25번 도로시설물 관리현황 정보의 검색결과 패턴을 비교하였을때 items에만 결과를 줄 것으로 예상
            BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
            if resultDf.empty: # 정상 데이터가 없는 경우
                print("NO DATA FOUND, CHECK CALS API")
                raise noDataFound()
            else:
                resultDfMerged = resultDfMerged.append(resultDf) 
                g_totalCount -= numOfRows
                print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
                if(g_totalCount <= 0):
                    break
                pageNo += 1
        
except noDataFound:
    print("exception handled")
    
        

i= 0
1 page scraping start
detail1is None !!!!
NO DATA FOUND, CHECK CALS API
exception handled


In [10]:
resultDfMerged.shape

(0, 0)

In [11]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

시설물 유지보수대장 목록 save compled


In [12]:
resultDfMerged.shape

(0, 0)